##  *Develop an agent that can achieve a specific goal and sub-goal in a maze 10X10 based on RL algorithm (Q-learning).*

In [35]:
# Import the necessary libraries to create the maze and give our agent the ability to randomly making choices

import numpy as np
import random

####  Setting up the environment

In [36]:
# Implemetation of the maze 10X10 as indicated in the assignment. 

# 1 represents the walls (unpassable)
# 0 represents an open path (passable)
# S marks the starting point of the agent
# G marks the sub-goal the agent should reach before continuing
# E marks the end goal

maze = np.array([[1,1,1,1,1,1,1,1,1,1],
                 [1,'S',1,0,0,0,1,0,0,1],
                 [1,0,1,0,1,0,1,0,1,1],
                 [1,0,0,0,1,0,0,0,0,1],
                 [1,1,1,0,1,1,1,1,0,1],
                 [1,0,1,'G',0,0,0,1,0,1],
                 [1,0,0,0,1,1,0,0,0,1],
                 [1,1,1,0,1,0,1,1,0,1],
                 [1,0,0,0,0,0,1,'E',0,1],
                 [1,1,1,1,1,1,1,1,1,1]], dtype=object)



# Implementatio of the Q-Table
# Considering the maze above, and the Q-learning algorithm, we need to initiate
# the Q-table. The Q-table include all possible states-actions pairs and their corresponing Q-values.

q_table = np.zeros((10,10,4))   # each states have 4 possible actions (up, down, left, right)
actions = ['up', 'down', 'left', 'right']

# Implementation of the agent's starting position in the maze, the sub-goal and the end-goal

start_position = (1,1)
sub_goal = (5,3)
end_goal = (8,7)


# Implementation of the reward system

reward_sub_goal = 10
reward_end_goal = 100
penalty = -1    

# Implementation of the agent's interactions with the environment, the way the agent moves in the maze
# and the way penalties and rewards are given to the agent.

def moves(state, actions):
    row, col = state

    if actions == 'up':
        next_state = (row - 1, col)
    elif actions == 'down':
        next_state = (row + 1, col)
    elif actions == 'left':
        next_state = (row, col - 1)
    elif actions == 'right':
        next_state = (row, col + 1)
    else:
        next_state = state

# Check if the next state is a wall or out of the maze
    if next_state[0] >= 0 and next_state[0] < 10 and next_state[1] >= 0 and next_state[1] < 10:
        if maze[next_state] != 1:
            if next_state == (5,3):
                return next_state, reward_sub_goal, False   #Sub-goal reached 
            elif next_state == (8,7):
                return next_state, reward_end_goal, True    #End-goal reached
            else:
                return next_state, penalty, False           #No goal reached, keep searching
        else:
            return state, penalty, False                    #Hit a wall, stay in the same state




# Implementation of the starting funtion, in order to restart the agent's position in the maze
# at the end of each episode.

def restarting():
    return start_position


# Implementation of the selection of the agent's action.
# The agent will randomly choose the actions to take in the Q-table, based on the epsilon-greedy policy.
# The action with the highest Q-value will be chosen 

def select_action(state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return np.random.randint(len(actions))
    else:
        return np.argmax(q_table[state[0], state[1]])

#### Implementation of the Q-learning algorithm

In [41]:
# Hyperparameters

alpha = 0.01  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Initial exploration rate. 
epsislon_variation = 0.95  # This parameter will decrease the epsilon value over time, in order to make the agent more greedy
min_epsilon = 0.01  # Minimum exploration rate reached by the agent
episodes = 1000 # Number of episodes the agent will run
steps = 1000  # Number of steps the agent will take in each episode

# Implementation of the Q-learning algorithm based on the mathematical formula
# Q(s,a) = Q(s,a) + alpha * (reward + gamma * max(Q(s',a')) - Q(s,a))

def q_value(state, action_index, reward, next_state):
    
    max_q_value = np.max(q_table[next_state[0], next_state[1]])
    actual_q_value = q_table[state[0], state[1], action_index]
    new_q_value = actual_q_value + alpha * (reward + gamma * max_q_value - actual_q_value)

    q_table[state[0], state[1], action_index] = new_q_value

#### Implementation of the training process

In [42]:
for episodes in range(episodes):
    
    state = restarting()
    done = False
    total_reward = 0

    for steps in range(steps):

        actions_index = select_action(state, epsilon)
        action = actions[actions_index]

        next_state, reward, done = moves(state, action)

        q_value(state, actions_index, reward, next_state)

        state
        total_reward += reward

        if done:
            break

    epsilon = max(min_epsilon, epsilon * epsislon_variation)

    if (episodes + 1) % 100 == 0:
        print(f'Episode {episodes + 1}/{episodes}, Total reward: {total_reward}')






Episode 100/99, Total reward: -901
Episode 200/199, Total reward: -801
Episode 300/299, Total reward: -701
Episode 400/399, Total reward: -601
Episode 500/499, Total reward: -501
Episode 600/599, Total reward: -401
Episode 700/699, Total reward: -301
Episode 800/799, Total reward: -201
Episode 900/899, Total reward: -101
Episode 1000/999, Total reward: -1
